In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numba import cuda

In [3]:
headlines=[]
is_sarcastic = []

file = open("../data/sarcasm_detection/Sarcasm_Headlines_Dataset_v2.json")

for line in file.readlines():
    is_sarcastic.append(json.loads(line).get('is_sarcastic'))
    headlines.append(json.loads(line).get('headline').strip())
    
    
print(len(headlines))
print(len(is_sarcastic))
    

28619
28619


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.utils import to_categorical

In [5]:
X_train,X_test,y_train,y_test = tts(headlines,is_sarcastic,test_size = 0.2,random_state = 101)

In [6]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [7]:
BATCH_SIZE = 16

tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index


In [8]:
vocab_size = len(word_index)
vocab_size

27722

In [9]:
pad_type = 'post'
trunc_type = 'post'
max_len = max([len(x) for x in X_train])
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences,padding=pad_type,truncating=trunc_type,maxlen=max_len)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences,padding=pad_type,truncating=trunc_type,maxlen=max_len)



In [10]:
max_len

926

In [11]:
vocab_size

27722

In [12]:
embed_dim = 16
vocab_size = vocab_size
input_len = max_len

In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size+1,embed_dim,input_length=input_len))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(max_len,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 926, 16)           443568    
_________________________________________________________________
dropout (Dropout)            (None, 926, 16)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 926, 128)          41472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 926)               60190     
_________________________________________________________________
dropout_1 (Dropout)          (None, 926)               0

In [15]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(train_padded,y_train,epochs=10,validation_data=(test_padded,y_test),verbose=1)

Epoch 1/10
716/716 [==============================] - 341s 476ms/step - loss: 0.4409 - accuracy: 0.7985 - val_loss: 0.3845 - val_accuracy: 0.8405
Epoch 2/10
716/716 [==============================] - 432s 603ms/step - loss: 0.2851 - accuracy: 0.8928 - val_loss: 0.3814 - val_accuracy: 0.8510
Epoch 3/10
716/716 [==============================] - 430s 600ms/step - loss: 0.1784 - accuracy: 0.9325 - val_loss: 0.3713 - val_accuracy: 0.8445
Epoch 4/10
716/716 [==============================] - 429s 600ms/step - loss: 0.1124 - accuracy: 0.9612 - val_loss: 0.4086 - val_accuracy: 0.8524
Epoch 5/10
368/716 [==============>...............] - ETA: 3:10 - loss: 0.0718 - accuracy: 0.9774

In [13]:
cuda.close()

CudaAPIError: [201] Call to cuDevicePrimaryCtxRelease results in CUDA_ERROR_INVALID_CONTEXT